# v2.2 exploration trying to make it work better

In [ ]:
# Environment
import os
import os.path as osp
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import sys
# Local modules
sys.path.append('..')
import reproducibility
import pandas as pd
from utils import print_dict_summary
from data_funcs import rmse, build_train_dict, combine_nested, subset_by_features
# from moisture_rnn import RNNParams, RNNData, RNN, RNN_LSTM, rnn_data_wrap
from moisture_rnn_pkl import pkl2train
from tensorflow.keras.callbacks import Callback
from utils import hash2
import copy
import logging
import pickle
from utils import logging_setup, read_yml, read_pkl, hash_ndarray, hash_weights, str2time
import yaml
import copy
import time

In [ ]:
logging_setup()

## Test Data

In [ ]:
file_paths = ['data/fmda_rocky_202403-05_f05.pkl']

In [ ]:
# Params used for data filtering
params_data = read_yml("params_data.yaml") 
params_data

In [ ]:
dat = read_pkl(file_paths[0])

In [ ]:
import importlib
import data_funcs
importlib.reload(data_funcs)
from data_funcs import build_train_dict

In [ ]:
params_data.update({'hours': None})

In [ ]:
train3 = build_train_dict(file_paths, params_data, spatial=False, forecast_step=3, drop_na=True)


## Test Other ML

In [ ]:
params = read_yml("params.yaml", subkey='xgb')
params

In [ ]:
dat = read_pkl("data/train.pkl")

In [ ]:
cases = [*dat.keys()]

In [ ]:
rnn_dat = RNNData(dat[cases[10]], params['scaler'], params['features_list'])
rnn_dat.train_test_split(
    time_fracs = [.8, .1, .1]
)
rnn_dat.scale_data()

In [ ]:
from moisture_models import XGB, RF, LM

In [ ]:
mod = XGB(params)

In [ ]:
mod.params

In [ ]:
mod.fit(rnn_dat.X_train, rnn_dat.y_train)

In [ ]:
preds = mod.predict(rnn_dat.X_test)

In [ ]:
rmse(preds, rnn_dat.y_test)

In [ ]:
plt.plot(rnn_dat.y_test)
plt.plot(preds)

In [ ]:
params = read_yml("params.yaml", subkey='rf')
rnn_dat = RNNData(dat[cases[10]], features_list = ['Ed', 'Ew', 'solar', 'wind', 'rain'])
rnn_dat.train_test_split(
    time_fracs = [.8, .1, .1]
)

In [ ]:
import importlib
import moisture_models
importlib.reload(moisture_models)

In [ ]:
params

In [ ]:
mod2 = RF(params)
mod2.fit(rnn_dat.X_train, rnn_dat.y_train.flatten())
preds2 = mod2.predict(rnn_dat.X_test)
print(rmse(preds2, rnn_dat.y_test.flatten()))
plt.plot(rnn_dat.y_test)
plt.plot(preds2)

In [ ]:
from moisture_models import RF
mod2 = RF(params)

In [ ]:
params = read_yml("params.yaml", subkey='lm')
rnn_dat = RNNData(dat[cases[10]], features_list = ['Ed', 'Ew', 'solar', 'wind', 'rain'])
rnn_dat.train_test_split(
    time_fracs = [.8, .1, .1]
)
mod = LM(params)

In [ ]:
mod.fit(rnn_dat.X_train, rnn_dat.y_train)
preds = mod.predict(rnn_dat.X_test)
print(rmse(preds2, rnn_dat.y_test.flatten()))

## Class RNN 

In [ ]:
from tensorflow.keras import layers,models

In [ ]:
import importlib
import moisture_rnn
importlib.reload(moisture_rnn)
from moisture_rnn import RNN, RNNParams

In [ ]:
params = RNNParams(read_yml("params.yaml", subkey='rnn'))

In [ ]:
params.update({
    'hidden_layers': ['lstm', 'conv1d', 'dense'],
    'hidden_units': [32, 32, 16],
    'hidden_activation': ['tanh', 'relu', 'relu'],
    'return_sequences': True
})

In [ ]:
import importlib
import moisture_rnn
importlib.reload(moisture_rnn)
from moisture_rnn import RNN, rnn_data_wrap

In [ ]:
reproducibility.set_seed()
mod = RNN(params)

In [ ]:
mod.model_train.summary()

In [ ]:
mod.model_predict.summary()

In [ ]:
rnn_dat = rnn_data_wrap(combine_nested(train3), params)

In [ ]:
params.update({
    'loc_batch_reset': rnn_dat.n_seqs # Used to reset hidden state when location changes for a given batch
})

In [ ]:
m, errs = mod.run_model(rnn_dat)

In [ ]:
errs.mean()

In [ ]:
params

## RNN

In [ ]:
import importlib 
import moisture_rnn
importlib.reload(moisture_rnn)
from moisture_rnn import RNN

In [ ]:
params = read_yml("params.yaml", subkey="lstm")
params = RNNParams(params)
params.update({
    'dense_layers': 2,
    'dense_units': 32
})

In [ ]:
params.update({'batch_schedule_type': 'step'})

In [ ]:
rnn_dat = rnn_data_wrap(combine_nested(train3), params)
reproducibility.set_seed(123)
rnn = RNN(params)

In [ ]:
rnn.model_train.summary()

In [ ]:
rnn.model_predict.summary()

In [ ]:
rnn.params

In [ ]:
rnn.model_train.fit(rnn_dat.X_train, rnn_dat.y_train, validation_data=(rnn_dat.X_val, rnn_dat.y_val), 
                    verbose=True, epochs=20)

In [ ]:
rnn.model_predict.set_weights(rnn.model_train.get_weights())

In [ ]:
rnn.model_predict.summary()

In [ ]:
rnn_dat.X_test.shape

In [ ]:
preds = rnn.model_predict.predict(rnn_dat.X_test)

In [ ]:
preds.shape

In [ ]:
rnn_dat.X_test.shape

In [ ]:
squared_diff = np.square(preds - rnn_dat.y_test)
mse = np.mean(squared_diff, axis=(1, 2))
errs = np.sqrt(mse)
errs.mean()

## Phys Initialized

In [ ]:
params.update({
    'epochs':100,
    'dense_layers': 0,
    'activation': ['relu', 'relu'],
    'phys_initialize': False,
    'dropout': [0,0],
    'space_fracs': [.8, .1, .1],
    'scaler': None
})

In [ ]:
import importlib
import moisture_rnn
importlib.reload(moisture_rnn)
from moisture_rnn import rnn_data_wrap

In [ ]:
params['scaler'] is None

In [ ]:
rnn_dat = rnn_data_wrap(combine_nested(train3), params)

In [ ]:
reproducibility.set_seed()
rnn = RNN(params)
m, errs = rnn.run_model(rnn_dat)

In [ ]:
rnn.model_train.summary()

In [ ]:
errs.mean()

In [ ]:
rnn_dat.X_train[:,:,0].mean()

In [ ]:
rnn_dat['features_list']

## LSTM

In [ ]:
import importlib 
import moisture_rnn
importlib.reload(moisture_rnn)
from moisture_rnn import RNN_LSTM

In [ ]:
params = read_yml("params.yaml", subkey="lstm")
params = RNNParams(params)

In [ ]:
rnn_dat = rnn_data_wrap(combine_nested(train3), params)

In [ ]:
params.update({
    'loc_batch_reset': rnn_dat.n_seqs # Used to reset hidden state when location changes for a given batch
})

In [ ]:
from moisture_rnn import ResetStatesCallback, EarlyStoppingCallback
params.update({'epochs': 50, 'learning_rate': 0.001, 'verbose_fit': True, 'rnn_layers': 2, 'rnn_units': 20, 'dense_layers': 1, 'dense_units': 10,
              'activation': ['tanh', 'tanh'], 'features_list': rnn_dat.features_list,
              'batch_schedule_type':'step', 'bmin': 10, 'bmax':rnn_dat.hours})
reproducibility.set_seed(123)
lstm = RNN_LSTM(params)

history = lstm.model_train.fit(rnn_dat.X_train, rnn_dat.y_train, 
                    batch_size = params['batch_size'], epochs=params['epochs'], 
                    callbacks = [ResetStatesCallback(params),
                                EarlyStoppingCallback(patience = 15)],
                   validation_data = (rnn_dat.X_val, rnn_dat.y_val))
              

In [ ]:
params = RNNParams(read_yml("params.yaml", subkey="lstm"))
params.update({'epochs': 50, 'learning_rate': 0.001, 'verbose_fit': True, 'rnn_layers': 2, 'rnn_units': 20, 'dense_layers': 1, 'dense_units': 10,
              'activation': ['tanh', 'tanh'], 'features_list': rnn_dat.features_list,
              'batch_schedule_type':'step', 'bmin': 10, 'bmax':rnn_dat.hours})
rnn_dat = rnn_data_wrap(combine_nested(train3), params)
params.update({
    'loc_batch_reset': rnn_dat.n_seqs # Used to reset hidden state when location changes for a given batch
})
reproducibility.set_seed(123)
lstm = RNN_LSTM(params)
m, errs = lstm.run_model(rnn_dat)

In [ ]:
errs.mean()

In [ ]:
params = RNNParams(read_yml("params.yaml", subkey="rnn"))
rnn_dat = rnn_data_wrap(combine_nested(train3), params)

In [ ]:
reproducibility.set_seed(123)
rnn = RNN(params)
m, errs = rnn.run_model(rnn_dat)

In [ ]:
errs.mean()